In [1]:
import shutup
shutup.please()

import pandas as pd
import numpy as np
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from sklearn import preprocessing
from datetime import datetime, date

In [2]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import plotly.express as px

### Directories

In [3]:
wheat_database = "/Users/hn/Documents/01_research_data/Other_people/Ehsan/wheat/"
data_dir = wheat_database + "data/"
separate_varieties_dir = data_dir + "varieties/"

In [4]:
###### List of variety names
csv_files = [x for x in os.listdir(separate_varieties_dir) if x.endswith(".xlsx")]

In [5]:
variety_names = [s.split(".")[0] for s in csv_files]
variety_names[:3]

['Diva', 'Whit', 'Ryan']

In [11]:
# all_varieties_grain_yields = pd.DataFrame()
# for a_file in csv_files:
#     df_ = pd.read_excel(separate_varieties_dir + a_file)
#     df_["variety"] = a_file.split(".")[0].lower()
#     all_varieties_grain_yields = pd.concat([all_varieties_grain_yields, df_])
    
# # change column names to lower case for consistency
# all_varieties_grain_yields.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
# all_varieties_grain_yields.head(2)

# filename = (separate_varieties_dir + "all_varieties_grainYields_stageAggregated.sav")

# export_ = {
#     "all_varieties_grain_yields": all_varieties_grain_yields,
#     "source_code": "corr_feat_select_separate_varieties",
#     "Author": "HN",
#     "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
# }

# pickle.dump(export_, open(filename, "wb"))

# all_varieties_grain_yields.head(2)

,location,latitude,longitude,year,emergence_precip,tillering_precip,jointing_precip,heading_precip,flowering_precip,grain_fill_precip,...,preseason_precip,preseason_temp,preseason_pdsi,preseason_spei,soil_type,slope,elevation,aspect,grain_yield,variety
0,Almira,47.51921,-118.53658,2008,2.5,4.3,12.9,0.5,1.8,0.0,...,123.9,1.241985,-2.178293,-2.42,6,7,596,142.765167,61.300000,diva
1,Almira,47.51921,-118.53658,2009,3.3,18.3,2.4,4.7,0.8,9.1,...,153.0,1.054403,-3.271750,-4.54,6,7,596,142.765167,56.933333,diva


In [8]:
filename = separate_varieties_dir + "all_varieties_grainYields_stageAggregated.sav"
all_varieties_GY = pd.read_pickle(filename)
all_varieties_GY = all_varieties_GY["all_varieties_grain_yields"]
all_varieties_GY.head(2)

,location,latitude,longitude,year,emergence_precip,tillering_precip,jointing_precip,heading_precip,flowering_precip,grain_fill_precip,...,preseason_precip,preseason_temp,preseason_pdsi,preseason_spei,soil_type,slope,elevation,aspect,grain_yield,variety
0,Almira,47.51921,-118.53658,2008,2.5,4.3,12.9,0.5,1.8,0.0,...,123.9,1.241985,-2.178293,-2.42,6,7,596,142.765167,61.300000,diva
1,Almira,47.51921,-118.53658,2009,3.3,18.3,2.4,4.7,0.8,9.1,...,153.0,1.054403,-3.271750,-4.54,6,7,596,142.765167,56.933333,diva


In [9]:
variety_names = list(all_varieties_GY["variety"].unique())
variety_names[0:3]

['diva', 'whit', 'ryan']

In [10]:
# Model for a given variety and later we do a for-loop or Kamiak parallel

a_variety = variety_names[0]
df_variety = all_varieties_GY[all_varieties_GY["variety"] == a_variety].copy()
df_variety.reset_index(drop=True, inplace=True)
df_variety.head(3)

,location,latitude,longitude,year,emergence_precip,tillering_precip,jointing_precip,heading_precip,flowering_precip,grain_fill_precip,...,preseason_precip,preseason_temp,preseason_pdsi,preseason_spei,soil_type,slope,elevation,aspect,grain_yield,variety
0,Almira,47.51921,-118.53658,2008,2.5,4.3,12.9,0.5,1.8,0.0,...,123.9,1.241985,-2.178293,-2.42,6,7,596,142.765167,61.300000,diva
1,Almira,47.51921,-118.53658,2009,3.3,18.3,2.4,4.7,0.8,9.1,...,153.0,1.054403,-3.271750,-4.54,6,7,596,142.765167,56.933333,diva
2,Almira,47.51921,-118.53658,2010,10.6,4.2,73.8,21.3,0.0,0.7,...,150.3,2.626170,-2.967105,-11.07,6,7,596,142.765167,46.566667,diva


In [10]:
list(df_variety.columns)

['location',
 'latitude',
 'longitude',
 'year',
 'emergence_precip',
 'tillering_precip',
 'jointing_precip',
 'heading_precip',
 'flowering_precip',
 'grain_fill_precip',
 'maturity_precip',
 'beyond_maturity_precip',
 'emergence_tmax',
 'tillering_tmax',
 'jointing_tmax',
 'heading_tmax',
 'flowering_tmax',
 'grain_fill_tmax',
 'maturity_tmax',
 'beyond_maturity_tmax',
 'emergence_tmin',
 'tillering_tmin',
 'jointing_tmin',
 'heading_tmin',
 'flowering_tmin',
 'grain_fill_tmin',
 'maturity_tmin',
 'beyond_maturity_tmin',
 'emergence_tavg',
 'tillering_tavg',
 'jointing_tavg',
 'heading_tavg',
 'flowering_tavg',
 'grain_fill_tavg',
 'maturity_tavg',
 'beyond_maturity_tavg',
 'emergence_gdd',
 'tillering_gdd',
 'jointing_gdd',
 'heading_gdd',
 'flowering_gdd',
 'grain_fill_gdd',
 'maturity_gdd',
 'beyond_maturity_gdd',
 'emergence_dgdd',
 'tillering_dgdd',
 'jointing_dgdd',
 'heading_dgdd',
 'flowering_dgdd',
 'grain_fill_dgdd',
 'maturity_dgdd',
 'beyond_maturity_dgdd',
 'emergence_d

In [11]:
df_variety.head(2)

,location,latitude,longitude,year,emergence_precip,tillering_precip,jointing_precip,heading_precip,flowering_precip,grain_fill_precip,...,preseason_precip,preseason_temp,preseason_pdsi,preseason_spei,soil_type,slope,elevation,aspect,grain_yield,variety
0,Almira,47.51921,-118.53658,2008,2.5,4.3,12.9,0.5,1.8,0.0,...,123.9,1.241985,-2.178293,-2.42,6,7,596,142.765167,61.300000,diva
1,Almira,47.51921,-118.53658,2009,3.3,18.3,2.4,4.7,0.8,9.1,...,153.0,1.054403,-3.271750,-4.54,6,7,596,142.765167,56.933333,diva


In [12]:
variable = "elevation"
a_years_data = df_variety[df_variety.year== 2008]
a_years_data[["location", variable]].groupby([variable]).count()

,location
elevation,
363,1
486,1
535,1
543,1
596,1
674,1
696,1
698,1
702,1


In [13]:
# df_variety[["location", "soil_type"]].groupby(["soil_type"]).count()

In [14]:
# df_variety[["location", "slope"]].groupby(["slope"]).count()

In [15]:
all_columns = sorted(list(df_variety.columns))
print (len(all_columns))
all_columns[0:3]

198


['aspect', 'beyond_maturity_#days', 'beyond_maturity_dgdd']

In [16]:
# some of these i do not like and some are constant (e.g. latitude). 
# what is the meaning of latitude's correlation with precipitation that changes over time?
# I donno
unwanted_features = ["year", "slope", "soil_type", 
                     "location", "latitude", "longitude", 
                     "aspect", "elevation", "variety"]
wanted_features = [x for x in all_columns if not (x in unwanted_features)]
wanted_features

['beyond_maturity_#days',
 'beyond_maturity_dgdd',
 'beyond_maturity_dtr',
 'beyond_maturity_etr',
 'beyond_maturity_fdd',
 'beyond_maturity_gdd',
 'beyond_maturity_hdd',
 'beyond_maturity_pdsi',
 'beyond_maturity_pet',
 'beyond_maturity_prdtr',
 'beyond_maturity_precip',
 'beyond_maturity_ravg',
 'beyond_maturity_rmax',
 'beyond_maturity_rmin',
 'beyond_maturity_soil',
 'beyond_maturity_spei',
 'beyond_maturity_srad',
 'beyond_maturity_tavg',
 'beyond_maturity_tmax',
 'beyond_maturity_tmin',
 'beyond_maturity_vpd',
 'beyond_maturity_vs',
 'beyond_maturity_wet_frequency',
 'emergence_#days',
 'emergence_dgdd',
 'emergence_dtr',
 'emergence_etr',
 'emergence_fdd',
 'emergence_gdd',
 'emergence_hdd',
 'emergence_pdsi',
 'emergence_pet',
 'emergence_prdtr',
 'emergence_precip',
 'emergence_ravg',
 'emergence_rmax',
 'emergence_rmin',
 'emergence_soil',
 'emergence_spei',
 'emergence_srad',
 'emergence_tavg',
 'emergence_tmax',
 'emergence_tmin',
 'emergence_vpd',
 'emergence_vs',
 'emerge

range(0, 11, 2)

In [17]:
traits = ["grain_yield"] # , "height", "protein"

In [59]:

len(a)

188

In [97]:
df_variety["variety"].unique()

array(['diva'], dtype=object)

In [101]:
all_outputs = {}

for features_corr_thresh in [0.5, .6, .7, .8]:
    for a_variety in variety_names:
        df_variety = all_varieties_GY[all_varieties_GY["variety"] == a_variety].copy()

        # Perform analysis for each trait
        for trait in traits:
            df = df_variety[wanted_features].copy() # Drop unwanted columns

            # Remove the other three traits (keep only the target trait)
            traits_to_remove = [t for t in traits if t != trait]
            df.drop(columns=traits_to_remove, inplace=True)

            # the rest of the code assumes target variable is at the end of DF
            df[trait] = df.pop(trait)

            # Compute the correlation matrix
            correlation_matrix = df.corr()

            # Identify pairs of highly correlated variables (correlation > features_corr_thresh)
            high_corr_pairs = []
            for i, col1 in enumerate(df.columns[:-1]): # Exclude the target column
                for col2 in df.columns[i+1:-1]:
                    if abs(correlation_matrix.loc[col1, col2]) > features_corr_thresh:
                        high_corr_pairs.append((col1, col2))

            # Decide which variable to remove based on correlation with target trait
            removed_features = set()
            for col1, col2 in high_corr_pairs:
                corr1 = abs(correlation_matrix.loc[col1, trait])
                corr2 = abs(correlation_matrix.loc[col2, trait])
                if corr1 < corr2:
                    removed_features.add(col1)
                else:
                    removed_features.add(col2)

            # Filter the DataFrame to remove the highly correlated variables
            filtered_df = df.drop(columns=list(removed_features))

            # Add year location back to the filtered DataFrame
            filtered_df = pd.concat([df_variety[unwanted_features], filtered_df], axis=1)

            filtered_df['environment'] = filtered_df['location'].astype(str) + '_' + \
                                                    filtered_df['year'].astype(str)
            
            key_ = f"{trait}_{a_variety}_corrFeatSelect_Thresh{int(features_corr_thresh*10)}"
            all_outputs[key_] = filtered_df
            
filename = separate_varieties_dir + f"{trait}_corrFeatSelect.sav"
export_ = {"all_varieties_differentThreshs": all_outputs,
           "source_code": "corr_feat_select_separate_varieties",
           "Author": "HN",
           "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

pickle.dump(export_, open(filename, "wb"))

In [39]:
filename = separate_varieties_dir + f"{trait}_corrFeatSelect_Thresh{int(features_corr_thresh*10)}.sav"
grain_yield_corrFeature = pd.read_pickle(filename)
grain_yield_corrFeature = grain_yield_corrFeature["filtered_df"]
grain_yield_corrFeature.head(2)

,year,slope,soil_type,location,latitude,longitude,aspect,elevation,variety,beyond_maturity_#days,...,tillering_pet,tillering_prdtr,tillering_ravg,tillering_rmin,tillering_spei,tillering_srad,tillering_tmin,tillering_vs,tillering_wet_frequency,environment
0,2008,7,6,Almira,47.51921,-118.53658,142.765167,596,diva,7.0,...,106.8,0.017651,52.986364,29.700,-0.578,269.327273,7.5900,3.531818,0.238095,Almira_2008
1,2009,7,6,Almira,47.51921,-118.53658,142.765167,596,diva,10.0,...,118.9,0.048111,52.514000,27.036,0.260,292.620000,4.7844,3.880000,0.208333,Almira_2009


In [40]:
grain_yield_corrFeature.shape

(172, 107)

In [66]:
trait

'grain_yield'

In [67]:
del(a)

In [4]:
#----------------------------#
#        HEADING DATE        #
#----------------------------#
# perform pairwise correlation analysis
df_env = pd.read_csv('df_22805_hd.csv')
df = df_env.drop(columns=['location', 'year'])

# Compute the correlation matrix
correlation_matrix = df.corr()

# Identify pairs of highly correlated variables (correlation > 0.6)
high_corr_pairs = []
for i, col1 in enumerate(df.columns[:-1]):  # Exclude the target column
    for col2 in df.columns[i+1:-1]:
        if abs(correlation_matrix.loc[col1, col2]) > 0.7:
            high_corr_pairs.append((col1, col2))

# Decide which variable to remove based on correlation with target
removed_features = set()
for col1, col2 in high_corr_pairs:
    corr1 = abs(correlation_matrix.loc[col1, 'Heading_date'])
    corr2 = abs(correlation_matrix.loc[col2, 'Heading_date'])
    if corr1 < corr2:
        removed_features.add(col1)
    else:
        removed_features.add(col2)

# Filter the DataFrame to remove the highly correlated variables
filtered_df = df.drop(columns=list(removed_features))
# Add year location back to the filtered DataFrame
filtered_df = pd.concat([df_env[drop_columns], filtered_df], axis=1)
# Add a new column named environment by combining location and year
filtered_df['environment'] = filtered_df['location'].astype(str) + '_' + filtered_df['year'].astype(str)
# filtered_df.to_csv('hd_33125.csv', index  = False)

removed_features

{'Emergence_#days',
 'Emergence_Tavg',
 'Emergence_dtr',
 'Emergence_etr',
 'Emergence_fdd',
 'Emergence_hdd',
 'Emergence_pdsi',
 'Emergence_precip',
 'Emergence_ravg',
 'Emergence_soil',
 'Emergence_tmax',
 'Emergence_tmin',
 'Emergence_vpd',
 'Emergence_wet_frequency',
 'Heading_#days',
 'Heading_Tavg',
 'Heading_dtr',
 'Heading_etr',
 'Heading_hdd',
 'Heading_prdtr',
 'Heading_precip',
 'Heading_ravg',
 'Heading_rmax',
 'Heading_soil',
 'Heading_srad',
 'Heading_tmax',
 'Heading_tmin',
 'Heading_vpd',
 'Jointing_#days',
 'Jointing_Tavg',
 'Jointing_dtr',
 'Jointing_etr',
 'Jointing_hdd',
 'Jointing_pdsi',
 'Jointing_prdtr',
 'Jointing_ravg',
 'Jointing_rmax',
 'Jointing_soil',
 'Jointing_srad',
 'Jointing_tmax',
 'Jointing_tmin',
 'Jointing_vpd',
 'Longitude',
 'Tillering_#days',
 'Tillering_Tavg',
 'Tillering_etr',
 'Tillering_hdd',
 'Tillering_pdsi',
 'Tillering_prdtr',
 'Tillering_precip',
 'Tillering_ravg',
 'Tillering_rmax',
 'Tillering_tmax',
 'Tillering_tmin',
 'Tillering_vp